<h1>2장 토큰과 임베딩</h1>
<i>LLM 구축의 핵심적인 부분인 토큰과 임베딩을 살펴 봅니다.</i>


<a href="https://github.com/rickiepark/handson-llm"><img src="https://img.shields.io/badge/GitHub%20Repository-black?logo=github"></a>
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rickiepark/handson-llm/blob/main/chapter02.ipynb)

---

이 노트북은 <[핸즈온 LLM](https://tensorflow.blog/handson-llm/)> 책 2장의 코드를 담고 있습니다.

---

<a href="https://tensorflow.blog/handson-llm/">
<img src="https://tensorflow.blog/wp-content/uploads/2025/05/ed95b8eca688ec98a8_llm.jpg" width="350"/></a>


---

💡 **NOTE**: 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.

---

In [ ]:
# Phi-3 모델과 호환성 때문에 transformers 4.48.3 버전을 사용합니다.
!pip install transformers==4.48.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 84.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.0
    Uninstalling transformers-4.55.0:
      Successfully uninstalled transformers-4.55.0


In [ ]:
# 깃허브에서 위젯 상태 오류를 피하기 위해 진행 표시줄을 나타내지 않도록 설정합니다.
from transformers.utils import logging

logging.disable_progress_bar()

# LLM 다운로드하고 실행하기

빠른 추론을 위해 GPU에 모델을 로드합니다. 모델과 토크나이저를 개별적으로 살펴 보기 위해 따로따로 로드합니다.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 모델과 토크나이저를 로드합니다.
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [1]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened.<|assistant|>"
prompt_kor = "사라에게 비극적인 정원사고에 대해 사과하는 이메일을 작성하고, 그것이 어떻게 일어났는지 설명하시오.<|assistant|>"

# 입력 프롬프트를 토큰으로 나눕니다.
input_ids = tokenizer(prompt_kor, return_tensors="pt").input_ids.to("cuda")

# 텍스트를 생성합니다.
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=20
)

# 출력을 프린트합니다.
print(tokenizer.decode(generation_output[0]))

NameError: name 'tokenizer' is not defined

In [ ]:
print(input_ids)

tensor([[14350,   385,  4876, 27746,  5281,   304, 19235,   363,   278, 25305,
           293, 16423,   292,   286,   728,   481, 29889, 12027,  7420,   920,
           372,  9559, 29889, 32001]], device='cuda:0')


In [ ]:
for id in input_ids[0]:
   print(tokenizer.decode(id))

Write
an
email
apolog
izing
to
Sarah
for
the
trag
ic
garden
ing
m
ish
ap
.
Exp
lain
how
it
happened
.
<|assistant|>


In [ ]:
generation_output

tensor([[14350,   385,  4876, 27746,  5281,   304, 19235,   363,   278, 25305,
           293, 16423,   292,   286,   728,   481, 29889, 12027,  7420,   920,
           372,  9559, 29889, 32001,  3323,   622, 29901,   317,  3742,   406,
          6225, 11763,   363,   278, 19906,   292,   341,   728,   481,    13,
            13,    13, 29928,   799]], device='cuda:0')

In [ ]:
print(tokenizer.decode(3323))
print(tokenizer.decode(622))
print(tokenizer.decode([3323, 622]))
print(tokenizer.decode(29901))

Sub
ject
Subject
:


# 훈련된 LLM 토큰나이저 비교하기


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

colors_list = [
    '102;194;165', '252;141;98', '141;160;203',
    '231;138;195', '166;216;84', '255;217;47'
]

def show_tokens(sentence, tokenizer_name):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    token_ids = tokenizer(sentence).input_ids
    for idx, t in enumerate(token_ids):
        # 텍스트를 인코딩한 후 다시 디코딩했을 때 원본 텍스트와 동일해지려면
        # clean_up_tokenization_spaces를 False로 지정해야 합니다.
        # 현재 이 매개변수의 기본값은 None(True에 해당)이며
        # transformers 4.45에서 True로 바뀔 예정입니다.
        # https://github.com/huggingface/transformers/issues/31884
        print(
            f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
            tokenizer.decode(t, clean_up_tokenization_spaces=False) +
            '\x1b[0m',
            end=' '
        )

In [ ]:
text = """
English and CAPITALIZATION
🎵 鸟
show_tokens False None elif == >= else: two tabs:"		" four spaces:"    "
12.0*50=600
"""

In [ ]:
show_tokens(text, "bert-base-uncased")

[CLS] english and capital ##ization [UNK] [UNK] show _ token ##s false none eli ##f = = > = else : two tab ##s : " " four spaces : " " 12 . 0 * 50 = 600 [SEP] 

In [ ]:
show_tokens(text, "bert-base-cased")

[CLS] English and CA ##PI ##TA ##L ##I ##Z ##AT ##ION [UNK] [UNK] show _ token ##s F ##als ##e None el ##if = = > = else : two ta ##bs : " " four spaces : " " 12 . 0 * 50 = 600 [SEP] 

In [ ]:
show_tokens(text, "gpt2")


 English  and  CAP ITAL IZ ATION 
 � � �  � � � 
 show _ t ok ens  False  None  el if  ==  >=  else :  two  tabs :" 	 	 "  four  spaces :"        " 
 12 . 0 * 50 = 600 
 

In [ ]:
show_tokens(text, "google/flan-t5-small")

English and CA PI TAL IZ ATION  <unk>  <unk> show _ to ken s Fal s e None  e l if = = > = else : two tab s : " " four spaces : " " 12. 0 * 50 = 600  </s> 

In [ ]:
# 공식 토크나이저는 `tiktoken`이지만 허깅 페이스 플랫폼에 동일한 토크나이저가 있습니다.
show_tokens(text, "Xenova/gpt-4")


 English  and  CAPITAL IZATION 
 � � �  � � � 
 show _tokens  False  None  elif  ==  >=  else :  two  tabs :" 	 	 "  four  spaces :"      "
 12 . 0 * 50 = 600 
 

In [ ]:
show_tokens(text, "bigcode/starcoder2-15b")


 English  and  CAPITAL IZATION 
 � � �   � � 
 show _ tokens  False  None  elif  ==  >=  else :  two  tabs :" 	 	 "  four  spaces :"      " 
 1 2 . 0 * 5 0 = 6 0 0 
 

In [ ]:
show_tokens(text, "facebook/galactica-1.3b")


 English  and  CAP ITAL IZATION 
 � � � �  � � � 
 show _ tokens  False  None  elif   ==   > =  else :  two  t abs : " 		 "  four  spaces : "      " 
 1 2 . 0 * 5 0 = 6 0 0 
 

In [ ]:
show_tokens(text, "microsoft/Phi-3-mini-4k-instruct")

 
 English and C AP IT AL IZ ATION 
 � � � �  � � � 
 show _ to kens False None elif == >= else : two tabs :" 	 	 " four spaces :"    " 
 1 2 . 0 * 5 0 = 6 0 0 
 

# (BERT와 같은) 언어 모델로 문맥을 고려한 단어 임베딩 만들기

In [ ]:
from transformers import AutoModel, AutoTokenizer

# 토크나이저를 로드합니다.
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

# 언어 모델을 로드합니다.
model = AutoModel.from_pretrained("microsoft/deberta-v3-xsmall")

# 문장을 토큰으로 나눕니다.
tokens = tokenizer('Hello world', return_tensors='pt')

# 토큰을 처리합니다.
output = model(**tokens)[0]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
output.shape

torch.Size([1, 4, 384])

In [ ]:
for token in tokens['input_ids'][0]:
    print(tokenizer.decode(token))

[CLS]
Hello
world
[SEP]


In [ ]:
output

tensor([[[-3.3060, -0.0507, -0.1098,  ..., -0.1704, -0.1618,  0.6932],
         [ 0.8918,  0.0740, -0.1583,  ...,  0.1869,  1.4760,  0.0751],
         [ 0.0871,  0.6364, -0.3050,  ...,  0.4729, -0.1829,  1.0157],
         [-3.1624, -0.1436, -0.0941,  ..., -0.0290, -0.1265,  0.7954]]],
       grad_fn=<NativeLayerNormBackward0>)

# 텍스트 임베딩 (문장과 전체 문서)

In [ ]:
from sentence_transformers import SentenceTransformer

# 모델을 로드합니다.
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# 텍스트를 텍스트 임베딩으로 변환합니다.
vector = model.encode("Best movie ever!")

In [ ]:
vector.shape

(768,)

# LLM 밖의 단어 임베딩


In [ ]:
# gensim을 설치합니다. 설치 후 코랩 런타임 세션을 다시 시작해 주세요.
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 14.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatibl

In [ ]:
import gensim.downloader as api

# 임베딩을 다운로드합니다 (66MB, glove, 위키백과에서 훈련됨, 벡터 크기: 50)
# "word2vec-google-news-300"도 선택 가능합니다.
# 더 자세한 옵션은 https://github.com/RaRe-Technologies/gensim-data 을 참고하세요.
model = api.load("glove-wiki-gigaword-50")

[==================================================] 100.0% 66.0/66.0MB downloaded


In [ ]:
model.most_similar([model['king']], topn=11)

[('king', 1.0000001192092896),
 ('prince', 0.8236179351806641),
 ('queen', 0.7839043140411377),
 ('ii', 0.7746230363845825),
 ('emperor', 0.7736247777938843),
 ('son', 0.766719400882721),
 ('uncle', 0.7627150416374207),
 ('kingdom', 0.7542161345481873),
 ('throne', 0.7539914846420288),
 ('brother', 0.7492411136627197),
 ('ruler', 0.7434253692626953)]

# 임베딩으로 노래 추천하기

In [ ]:
import pandas as pd
from urllib import request

# 재생목록 데이터셋 파일을 가져옵니다.
data = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt')

# 재생목록 파일을 파싱합니다. 처음 두 줄은 메타데이터만 담고 있으므로 건너뜁니다.
lines = data.read().decode("utf-8").split('\n')[2:]

# 하나의 노래만 있는 재생목록은 삭제합니다.
playlists = [s.rstrip().split() for s in lines if len(s.split()) > 1]

# 노래의 메타데이터를 로드합니다.
songs_file = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt')
songs_file = songs_file.read().decode("utf-8").split('\n')
songs = [s.rstrip().split('\t') for s in songs_file]
songs_df = pd.DataFrame(data=songs, columns = ['id', 'title', 'artist'])
songs_df = songs_df.set_index('id')

In [ ]:
print('재생목록 #1:\n ', playlists[0], '\n')
print('재생목록 #2:\n ', playlists[1])

재생목록 #1:
  ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '2', '42', '43', '44', '45', '46', '47', '48', '20', '49', '8', '50', '51', '52', '53', '54', '55', '56', '57', '25', '58', '59', '60', '61', '62', '3', '63', '64', '65', '66', '46', '47', '67', '2', '48', '68', '69', '70', '57', '50', '71', '72', '53', '73', '25', '74', '59', '20', '46', '75', '76', '77', '59', '20', '43'] 

재생목록 #2:
  ['78', '79', '80', '3', '62', '81', '14', '82', '48', '83', '84', '17', '85', '86', '87', '88', '74', '89', '90', '91', '4', '73', '62', '92', '17', '53', '59', '93', '94', '51', '50', '27', '95', '48', '96', '97', '98', '99', '100', '57', '101', '102', '25', '103', '3', '104', '105', '106', '107', '47', '108', '109', '110', '111', '112', '113', '25', '63', '62', '114', '115', '84', '116', '117', '118', 

In [ ]:
from gensim.models import Word2Vec

# Word2Vec 모델을 훈련합니다.
model = Word2Vec(
    playlists, vector_size=32, window=20, negative=50, min_count=1, workers=4
)

In [ ]:
song_id = 2172

# 노래 ID 2172와 비슷한 노래를 찾으라고 모델에게 요청합니다.
model.wv.most_similar(positive=str(song_id))

[('2849', 0.9978032112121582),
 ('20010', 0.9973780512809753),
 ('5586', 0.9962414503097534),
 ('3094', 0.9962045550346375),
 ('5634', 0.995397686958313),
 ('1922', 0.99519282579422),
 ('3119', 0.9950017333030701),
 ('6658', 0.9949158430099487),
 ('6624', 0.9946021437644958),
 ('10105', 0.9938193559646606)]

In [ ]:
print(songs_df.iloc[2172])

title     Fade To Black
artist        Metallica
Name: 2172 , dtype: object


In [ ]:
import numpy as np

def print_recommendations(song_id):
    similar_songs = np.array(
        model.wv.most_similar(positive=str(song_id),topn=5)
    )[:,0]
    return  songs_df.iloc[similar_songs]

# 추천 노래 출력
print_recommendations(2172)

,title,artist
id,,
2849,Run To The Hills,Iron Maiden
20010,Let It Die,Ozzy Osbourne
5586,The Last In Line,Dio
3094,Breaking The Law,Judas Priest
5634,Mr. Brownstone,Guns N' Roses


In [ ]:
print_recommendations(2172)

,title,artist
id,,
2849,Run To The Hills,Iron Maiden
20010,Let It Die,Ozzy Osbourne
5586,The Last In Line,Dio
3094,Breaking The Law,Judas Priest
5634,Mr. Brownstone,Guns N' Roses


In [ ]:
print_recommendations(842)

,title,artist
id,,
413,If I Ruled The World (Imagine That) (w\/ Laury...,Nas
5788,Drop It Like It's Hot (w\/ Pharrell),Snoop Dogg
12205,Give It Up To Me,Sean Paul
374,Jook Gal,Elephant Man
5668,How We Do (w\/ 50 Cent),The Game
